# JsonOutputParser

이 출력 파서는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주며, 그 스키마에 맞게 LLM에서 데이터를 조회하여 결과를 도출해줍니다.

LLM이 데이터를 정확하고 효율적으로 처리하여 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량이 충분해야 한다는 점을 기억해야 합니다.

데이터 모델을 정의할 때는 Pydantic과 같은 도구를 사용하여, 스키마가 잘 정의되고 검증될 수 있도록 할 수 있습니다.

이렇게 하면 사용자가 특정 데이터 구조를 기반으로 정보를 요청하고, 모델이 그 요구사항에 맞는 데이터를 정확하게 제공할 수 있게 됩니다.


In [1]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화 및 온도 설정

원하는 출력 구조를 정의합니다.


In [3]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="Concise description about topic")
    hashtags: str = Field(description="Some keywords in hashtag format")

- `JsonOutputParser`를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입합니다.
- `PromptTemplate`을 이용하여 사용자 질의에 대한 답변을 생성하는 템플릿을 구성합니다. 이때, 질의(`query`)와 포맷 지시사항(`format_instructions`)을 변수로 사용합니다.
- `prompt`, `model`, `parser`를 연결하여 처리 체인을 구성합니다.
- 처리 체인을 호출하여 `query`에 대한 응답을 생성합니다.


In [6]:
# 질의 작성
query = "온난화에 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

prompt = PromptTemplate(
    # 사용자 쿼리에 답하십시오.
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    # 부분 변수에 형식 지시사항 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"query": query})  # 체인을 호출하여 쿼리 실행

{'description': '온난화는 지구 온도가 상승하는 현상을 의미하며, 이는 대기 중 온실가스 농도 증가와 인간 활동에 의해 발생합니다. 이로 인해 극지방의 빙하가 녹아 해수면 상승, 기후 변화 등의 문제가 발생할 수 있습니다.',
 'hashtags': '#온난화 #기후변화 #온실가스'}

## Without Pydantic

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.


- `JsonOutputParser` 클래스의 인스턴스를 `parser` 변수에 할당합니다.
- `PromptTemplate` 클래스를 사용하여 사용자 쿼리에 대한 프롬프트를 생성합니다. 이때, `template` 매개변수에는 사용자 쿼리를 처리할 템플릿 문자열을, `input_variables`에는 템플릿에서 사용할 변수 목록을, `partial_variables`에는 `parser`에서 제공하는 형식 지침을 포함합니다.
- `prompt`, `model`, `parser`를 연결하여 `chain`을 구성합니다.
- `chain.invoke` 메소드를 호출하여 `{"query": query}` 매개변수를 전달함으로써, 사용자 쿼리에 대한 처리를 시작합니다.


In [10]:
# 질의 작성
query = "온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

parser = JsonOutputParser()  # JSON 출력 파서 초기화

prompt = PromptTemplate(
    # 사용자 쿼리에 답변하는 템플릿
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수로 'query' 사용
    # 부분 변수로 포맷 지시사항 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser  # 프롬프트, 모델, 파서를 연결하는 체인 생성

chain.invoke({"query": query})  # 체인을 호출하여 농담 쿼리 처리

{'description': '온난화는 지구 온도가 상승하는 현상을 의미합니다. 이는 대기 중 온실가스 농도 증가로 인해 발생하며, 지구 기후와 생태계에 심각한 영향을 미칩니다.',
 'hashtags': ['온난화', '기후변화', '지구온난화', '온실가스']}